![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# 🧠 Section 3, Notebook 3: Memory Management - Handling Long Conversations

**⏱️ Estimated Time:** 50-60 minutes

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Understand** why long conversations need management (token limits, cost, performance)
2. **Implement** conversation summarization to preserve key information
3. **Build** context compression strategies (truncation, priority-based, summarization)
4. **Configure** automatic memory management with Agent Memory Server
5. **Decide** when to apply each technique based on conversation characteristics

---

## 🔗 Where We Are

### **Your Journey So Far:**

**Section 3, Notebook 1:** Memory Fundamentals
- ✅ Working memory for conversation continuity
- ✅ Long-term memory for persistent knowledge
- ✅ The grounding problem and reference resolution
- ✅ Memory types (semantic, episodic, message)

**Section 3, Notebook 2:** Memory-Enhanced RAG
- ✅ Integrated all four context types
- ✅ Built complete memory-enhanced RAG system
- ✅ Demonstrated benefits of stateful conversations

**Your memory system works!** It can:
- Remember conversation history across turns
- Store and retrieve long-term facts
- Resolve references ("it", "that course")
- Provide personalized recommendations

### **But... What About Long Conversations?**

**Questions we can't answer yet:**
- ❓ What happens when conversations get really long?
- ❓ How do we handle token limits?
- ❓ How much does a 50-turn conversation cost?
- ❓ Can we preserve important context while reducing tokens?
- ❓ When should we summarize vs. truncate vs. keep everything?

---

## 🚨 The Long Conversation Problem

Before diving into solutions, let's understand the fundamental problem.

### **The Problem: Unbounded Growth**

Every conversation turn adds messages to working memory:

```
Turn 1:  System (500) + Messages (200) = 700 tokens ✅
Turn 5:  System (500) + Messages (1,000) = 1,500 tokens ✅
Turn 20: System (500) + Messages (4,000) = 4,500 tokens ✅
Turn 50: System (500) + Messages (10,000) = 10,500 tokens ⚠️
Turn 100: System (500) + Messages (20,000) = 20,500 tokens ⚠️
Turn 200: System (500) + Messages (40,000) = 40,500 tokens ❌
```

**Without management, conversations grow unbounded!**

### **Why This Matters**

**1. Token Limits (Hard Constraint)**
- GPT-4o: 128K tokens (~96,000 words)
- GPT-3.5: 16K tokens (~12,000 words)
- Eventually, you'll hit the limit and conversations fail

**2. Cost (Economic Constraint)**
- Input tokens cost money  (e.g. $0.0025 /  1K  tokens for GPT-4o)

- A 50-turn conversation = ~10,000 tokens = $0.025 per query

- Over 1,000 conversations = $25 just for conversation history!

**3. Performance (Quality Constraint)**
- More tokens = longer processing time
- Context Rot: LLMs struggle with very long contexts
- Important information gets "lost in the middle"

**4. User Experience**
- Slow responses frustrate users
- Expensive conversations aren't sustainable
- Failed conversations due to token limits are unacceptable

### **The Solution: Memory Management**

We need strategies to:
- ✅ Keep conversations within token budgets
- ✅ Preserve important information
- ✅ Maintain conversation quality
- ✅ Control costs
- ✅ Enable indefinite conversations

---

## 📦 Part 0: Setup and Environment

Let's set up our environment and create tools for measuring conversation growth.

### ⚠️ Prerequisites

**Before running this notebook, make sure you have:**

1. **Docker Desktop running** - Required for Redis and Agent Memory Server

2. **Environment variables** - Create a `.env` file in the `reference-agent` directory:
   ```bash
   # Copy the example file
   cd ../../reference-agent
   cp .env.example .env

   # Edit .env and add your OpenAI API key
   # OPENAI_API_KEY=your_actual_openai_api_key_here
   ```

3. **Run the setup script** - This will automatically start Redis and Agent Memory Server:
   ```bash
   cd ../../reference-agent
   python setup_agent_memory_server.py
   ```


---


### Automated Setup Check

Let's run the setup script to ensure all services are running properly.


In [1]:
# Run the setup script to ensure Redis and Agent Memory Server are running
import subprocess
import sys
from pathlib import Path

# Path to setup script
setup_script = Path("../../reference-agent/setup_agent_memory_server.py")

if setup_script.exists():
    print("Running automated setup check...\n")
    result = subprocess.run(
        [sys.executable, str(setup_script)],
        capture_output=True,
        text=True
    )
    print(result.stdout)
    if result.returncode != 0:
        print("⚠️  Setup check failed. Please review the output above.")
        print(result.stderr)
    else:
        print("\n✅ All services are ready!")
else:
    print("⚠️  Setup script not found. Please ensure services are running manually.")


Running automated setup check...


🔧 Agent Memory Server Setup
📊 Checking Redis...
✅ Redis is running
📊 Checking Agent Memory Server...
🔍 Agent Memory Server container exists. Checking health...
✅ Agent Memory Server is running and healthy
✅ No Redis connection issues detected

✅ Setup Complete!
📊 Services Status:
   • Redis: Running on port 6379
   • Agent Memory Server: Running on port 8088

🎯 You can now run the notebooks!


✅ All services are ready!


---


### Install Dependencies

If you haven't already installed the reference-agent package, uncomment and run the following:


In [2]:
# Uncomment to install reference-agent package
# %pip install -q -e ../../reference-agent

# Uncomment to install agent-memory-client
# %pip install -q agent-memory-client


### Import Dependencies


In [3]:
# Standard library imports
import os
import time
import asyncio
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, field
from datetime import datetime
from pathlib import Path

# LangChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage

# Redis and Agent Memory
from agent_memory_client import MemoryAPIClient, MemoryClientConfig
from agent_memory_client.models import WorkingMemory, MemoryMessage, ClientMemoryRecord

# Token counting
import tiktoken

# For visualization
from collections import defaultdict

print("✅ All imports successful")


✅ All imports successful


### Load Environment Variables


In [4]:
from dotenv import load_dotenv

# Load environment variables from reference-agent directory
env_path = Path("../../reference-agent/.env")
load_dotenv(dotenv_path=env_path)

# Verify required environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
AGENT_MEMORY_URL = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")

if not OPENAI_API_KEY:
    print(f"""❌ OPENAI_API_KEY not found!

Please create a .env file at: {env_path.absolute()}

With the following content:
OPENAI_API_KEY=your_openai_api_key
REDIS_URL=redis://localhost:6379
AGENT_MEMORY_URL=http://localhost:8088
""")
else:
    print("✅ Environment variables configured")
    print(f"   Redis URL: {REDIS_URL}")
    print(f"   Agent Memory URL: {AGENT_MEMORY_URL}")


✅ Environment variables configured
   Redis URL: redis://localhost:6379
   Agent Memory URL: http://localhost:8088


### Initialize Clients


In [5]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7
)

# Initialize embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

# Initialize Agent Memory Client
memory_config = MemoryClientConfig(base_url=AGENT_MEMORY_URL)
memory_client = MemoryAPIClient(config=memory_config)

# Initialize tokenizer for counting
tokenizer = tiktoken.encoding_for_model("gpt-4o")

def count_tokens(text: str) -> int:
    """Count tokens in text using tiktoken."""
    return len(tokenizer.encode(text))

print("✅ Clients initialized")
print(f"   LLM: {llm.model_name}")
print(f"   Embeddings: text-embedding-3-small")
print(f"   Memory Server: {AGENT_MEMORY_URL}")


✅ Clients initialized
   LLM: gpt-4o
   Embeddings: text-embedding-3-small
   Memory Server: http://localhost:8088


---

## 📊 Part 1: Understanding Conversation Growth

Let's visualize how conversations grow and understand the implications.


### 🔬 Research Context: Why Context Management Matters

Modern LLMs have impressive context windows:
- **GPT-4o**: 128K tokens (~96,000 words)
- **Claude 3.5**: 200K tokens (~150,000 words)
- **Gemini 1.5 Pro**: 1M tokens (~750,000 words)

**But here's the problem:** Larger context windows don't guarantee better performance.

#### The "Lost in the Middle" Problem

Research by Liu et al. (2023) in their paper ["Lost in the Middle: How Language Models Use Long Contexts"](https://arxiv.org/abs/2307.03172) revealed critical findings:

**Key Finding #1: U-Shaped Performance**
- Models perform best when relevant information is at the **beginning** or **end** of context
- Performance **significantly degrades** when information is in the **middle** of long contexts
- This happens even with models explicitly designed for long contexts

**Key Finding #2: Non-Uniform Degradation**
- It's not just about hitting token limits
- Quality degrades **even within the context window**
- The longer the context, the worse the "middle" performance becomes

**Key Finding #3: More Context ≠ Better Results**
- In some experiments, GPT-3.5 performed **worse** with retrieved documents than with no documents at all
- Adding more context can actually **hurt** performance if not managed properly

**Why This Matters for Memory Management:**
- Simply storing all conversation history isn't optimal
- We need **intelligent compression** to keep important information accessible
- **Position matters**: Recent context (at the end) is naturally well-positioned
- **Quality over quantity**: Better to have concise, relevant context than exhaustive history

**References:**
- Liu, N. F., Lin, K., Hewitt, J., Paranjape, A., Bevilacqua, M., Petroni, F., & Liang, P. (2023). Lost in the Middle: How Language Models Use Long Contexts. *Transactions of the Association for Computational Linguistics (TACL)*.


### Demo 1: Token Growth Over Time

Now let's see this problem in action by simulating conversation growth.

#### Step 1: Define our system prompt and count its tokens


In [6]:
# System prompt (constant across all turns)
system_prompt = """You are a helpful course advisor for Redis University.
Help students find courses, check prerequisites, and plan their schedule.
Be friendly, concise, and accurate."""

system_tokens = count_tokens(system_prompt)

print(f"System prompt: {system_tokens} tokens")


System prompt: 31 tokens


#### Step 2: Simulate how tokens grow with each conversation turn


In [7]:
# Assume average message pair (user + assistant) = 100 tokens
avg_message_pair_tokens = 100

print("\nConversation Growth Simulation:")
print("=" * 80)
print(f"{'Turn':<8} {'Messages':<10} {'Conv Tokens':<15} {'Total Tokens':<15} {'Cost ($)':<12}")
print("-" * 80)

for turn in [1, 5, 10, 20, 30, 50, 75, 100, 150, 200]:
    # Each turn = user message + assistant message
    num_messages = turn * 2
    conversation_tokens = num_messages * (avg_message_pair_tokens // 2)
    total_tokens = system_tokens + conversation_tokens

    # Cost calculation (GPT-4o input: $0.0025 per 1K tokens)
    cost_per_query = (total_tokens / 1000) * 0.0025

    # Visual indicator
    if total_tokens < 5000:
        indicator = "✅"
    elif total_tokens < 20000:
        indicator = "⚠️"
    else:
        indicator = "❌"

    print(f"{turn:<8} {num_messages:<10} {conversation_tokens:<15,} {total_tokens:<15,} ${cost_per_query:<11.4f} {indicator}")



Conversation Growth Simulation:
Turn     Messages   Conv Tokens     Total Tokens    Cost ($)    
--------------------------------------------------------------------------------
1        2          100             131             $0.0003      ✅
5        10         500             531             $0.0013      ✅
10       20         1,000           1,031           $0.0026      ✅
20       40         2,000           2,031           $0.0051      ✅
30       60         3,000           3,031           $0.0076      ✅
50       100        5,000           5,031           $0.0126      ⚠️
75       150        7,500           7,531           $0.0188      ⚠️
100      200        10,000          10,031          $0.0251      ⚠️
150      300        15,000          15,031          $0.0376      ⚠️
200      400        20,000          20,031          $0.0501      ❌


### Demo 2: Cost Analysis

Let's calculate the cumulative cost of long conversations.

**Why costs grow quadratically:**
- Turn 1: Process 100 tokens
- Turn 2: Process 200 tokens (includes turn 1)
- Turn 3: Process 300 tokens (includes turns 1 & 2)
- Turn N: Process N×100 tokens

Total cost = 100 + 200 + 300 + ... + N×100 = **O(N²)** growth!

#### Step 1: Create a function to calculate conversation costs


In [8]:
def calculate_conversation_cost(num_turns: int, avg_tokens_per_turn: int = 100) -> Dict[str, float]:
    """
    Calculate cost metrics for a conversation.

    Args:
        num_turns: Number of conversation turns
        avg_tokens_per_turn: Average tokens per turn (user + assistant)

    Returns:
        Dictionary with cost metrics
    """
    system_tokens = 50  # Simplified

    # Cumulative cost (each turn includes all previous messages)
    cumulative_tokens = 0
    cumulative_cost = 0.0

    for turn in range(1, num_turns + 1):
        # Total tokens for this turn
        conversation_tokens = turn * avg_tokens_per_turn
        total_tokens = system_tokens + conversation_tokens

        # Cost for this turn (input tokens)
        turn_cost = (total_tokens / 1000) * 0.0025
        cumulative_cost += turn_cost
        cumulative_tokens += total_tokens

    return {
        "num_turns": num_turns,
        "final_tokens": system_tokens + (num_turns * avg_tokens_per_turn),
        "cumulative_tokens": cumulative_tokens,
        "cumulative_cost": cumulative_cost,
        "avg_cost_per_turn": cumulative_cost / num_turns
    }

print("✅ Cost calculation function defined")


✅ Cost calculation function defined


#### Step 2: Compare costs across different conversation lengths


In [12]:
print("Cost Analysis for Different Conversation Lengths:")
print("=" * 80)
print(f"{'Turns':<10} {'Final Tokens':<15} {'Cumulative Tokens':<20} {'Total Cost':<15} {'Avg/Turn'}")
print("-" * 80)

for num_turns in [10, 25, 50, 100, 200]:
    metrics = calculate_conversation_cost(num_turns)
    print(f"{metrics['num_turns']:<10} "
          f"{metrics['final_tokens']:<15,} "
          f"{metrics['cumulative_tokens']:<20,} "
          f"${metrics['cumulative_cost']:<14.2f} "
          f"${metrics['avg_cost_per_turn']:.4f}")


Cost Analysis for Different Conversation Lengths:
Turns      Final Tokens    Cumulative Tokens    Total Cost      Avg/Turn
--------------------------------------------------------------------------------
10         1,050           6,000                $0.02           $0.0015
25         2,550           33,750               $0.08           $0.0034
50         5,050           130,000              $0.33           $0.0065
100        10,050          510,000              $1.27           $0.0127
200        20,050          2,020,000            $5.05           $0.0253


#### Key Takeaways

**Without memory management:**
- Costs grow **quadratically** (O(N²))
  
- A 100-turn conversation costs ~$1.50 in total

  
- A 200-turn conversation costs ~$6.00 in total

- At scale (1000s of users), this becomes unsustainable

**The solution:** Intelligent memory management to keep conversations within budget while preserving quality.


---

## 🎯 Part 2: Context Summarizaton

**Context summarization** is the process of condensing conversation history into a compact representation that preserves essential information while dramatically reducing token count.

Picture a chat assistant helping someone plan a wedding over 50 messages:
- It captures the critical stuff: venue choice, budget, guest count, vendor decisions
- It grabs the decisions and ditches the small talk
- Later messages can reference "the venue we picked" without replaying the entire debate
  
**Same deal with LLM chats:**
- Squash ancient messages into a tight little paragraph
- Keep the gold (facts, choices, what the user loves/hates)
- Leave fresh messages untouched (they're still doing work)
- Slash token usage by 50-80% without lobotomizing the conversation

### Why Should You Care About Summarization?

Summarization tackles three gnarly problems:

**1. Plays Nice With Token Caps (Callback to Part 1)**
- Chats balloon up forever if you let them
- Summarization keeps you from hitting the ceiling
- **Real talk:** 50 messages (10K tokens) → Compressed summary + 4 fresh messages (2.5K tokens)

**2. Fixes the Context Rot Problem (Also From Part 1)**
- Remember that "Lost in the Middle" mess? Old info gets buried and ignored
- Summarization yanks that old stuff to the front in condensed form
- Fresh messages chill at the end (where the model actually pays attention)
- **Upshot:** Model performs better AND you save space—win-win

**3. Keeps Working Memory From Exploding (Throwback to Notebook 1)**
- Working memory = your conversation backlog
- Without summarization, it just keeps growing like a digital hoarder's closet
- Summarization gives it a haircut regularly
- **Payoff:** Conversations that can actually go the distance

### When Should You Reach for This Tool?

**Great for:**
- ✅ Marathon conversations (10+ back-and-forths)
- ✅ Chats that have a narrative arc (customer support, coaching sessions)
- ✅ Situations where you want history but not ALL the history
- ✅ When the recent stuff matters most

**Skip it when:**
- ❌ Quick exchanges (under 5 turns—don't overthink it)
- ❌ Every syllable counts (legal docs, medical consultations)
- ❌ You might need verbatim quotes from way back
- ❌ The extra LLM call for summarization costs too much time or money

### Where Summarization Lives in Your Memory Stack
```
┌─────────────────────────────────────────────────────────┐
│                  Your LLM Agent Brain                   │
│                                                         │
│  Context Window (128K tokens available)                 │
│  ┌────────────────────────────────────────────────┐     │
│  │ 1. System Prompt (500 tokens)                  │     │
│  │ 2. Long-term Memory Bank (1,000 tokens)        │     │
│  │ 3. RAG Retrieval Stuff (2,000 tokens)          │     │
│  │ 4. Working Memory Zone:                        │     │
│  │    ┌──────────────────────────────────────┐    │     │
│  │    │ [COMPRESSED HISTORY] (500 tokens)    │    │     │
│  │    │ - Critical facts from rounds 1-20    │    │     │
│  │    │ - Decisions that were locked in      │    │     │
│  │    │ - User quirks and preferences        │    │     │
│  │    └──────────────────────────────────────┘    │     │
│  │    Live Recent Messages (1,000 tokens)         │     │
│  │    - Round 21: User shot + Assistant reply     │     │
│  │    - Round 22: User shot + Assistant reply     │     │
│  │    - Round 23: User shot + Assistant reply     │     │
│  │    - Round 24: User shot + Assistant reply     │     │
│  │ 5. Current Incoming Query (200 tokens)         │     │
│  └────────────────────────────────────────────────┘     │
│                                                         │
│  Running total: ~5,200 tokens (instead of 15K—nice!)    │
└─────────────────────────────────────────────────────────┘
```

#### The Bottom Line: 
Summarization is a *compression technique* for working memory that maintains conversation continuity while keeping token counts manageable.

### 🔬 Research Foundation: Recursive Summarization

Wang et al. (2023) in ["Recursively Summarizing Enables Long-Term Dialogue Memory in Large Language Models"](https://arxiv.org/abs/2308.15022) demonstrated that:

**Key Insight:** Recursive summarization enables LLMs to handle extremely long conversations by:
1. Memorizing small dialogue contexts
2. Recursively producing new memory using previous memory + new contexts
3. Maintaining consistency across long conversations

**Their findings:**
- Improved response consistency in long-context conversations
- Works well with both long-context models (8K, 16K) and retrieval-enhanced LLMs
- Provides a practical solution for modeling extremely long contexts

**Practical Application:**
- Summarize old messages while keeping recent ones intact
- Preserve key information (facts, decisions, preferences)
- Compress redundant or less important information

**References:**
- Wang, Q., Fu, Y., Cao, Y., Wang, S., Tian, Z., & Ding, L. (2023). Recursively Summarizing Enables Long-Term Dialogue Memory in Large Language Models. *Neurocomputing* (Accepted).


### Theory: What to Preserve vs. Compress

When summarizing conversations, we need to be strategic about what to keep and what to compress.

**What to Preserve:**
- ✅ Key facts and decisions
- ✅ Student preferences and goals
- ✅ Important course recommendations
- ✅ Prerequisites and requirements
- ✅ Recent context (last few messages)

**What to Compress:**
- 📦 Small talk and greetings
- 📦 Redundant information
- 📦 Old conversation details
- 📦 Resolved questions

**When to Summarize:**
- Token threshold exceeded (e.g., > 2000 tokens)
- Message count threshold exceeded (e.g., > 10 messages)
- Time-based (e.g., after 1 hour)
- Manual trigger


### Building Summarization Step-by-Step

Let's build our summarization system incrementally, starting with simple components.

#### Step 1: Create a data structure for conversation messages

**What we're building:** A data structure to represent individual messages with metadata.

**Why it's needed:** We need to track not just the message content, but also:
- Who sent it (user, assistant, system)
- When it was sent (timestamp)
- How many tokens it uses (for threshold checks)

**How it works:** Python's `@dataclass` decorator creates a clean, type-safe structure with automatic initialization and token counting.


In [13]:
@dataclass
class ConversationMessage:
    """Represents a single conversation message."""
    role: str  # "user", "assistant", "system"
    content: str
    timestamp: float = field(default_factory=time.time)
    token_count: Optional[int] = None

    def __post_init__(self):
        if self.token_count is None:
            self.token_count = count_tokens(self.content)

print("✅ ConversationMessage dataclass defined")

# Test it
test_msg = ConversationMessage(
    role="user",
    content="What courses do you recommend for machine learning?"
)
print(f"\nExample message:")
print(f"  Role: {test_msg.role}")
print(f"  Content: {test_msg.content}")
print(f"  Tokens: {test_msg.token_count}")


✅ ConversationMessage dataclass defined

Example message:
  Role: user
  Content: What courses do you recommend for machine learning?
  Tokens: 9


#### Step 2: Create a function to check if summarization is needed

**What we're building:** A decision function that determines when to trigger summarization.

**Why it's needed:** We don't want to summarize too early (loses context) or too late (hits token limits). We need smart thresholds.

**How it works:**
- Checks if we have enough messages to make summarization worthwhile
- Calculates total token count across all messages
- Returns `True` if either threshold (tokens OR messages) is exceeded
- Ensures we keep at least `keep_recent` messages unsummarized

**When to summarize:**
- Token threshold: Prevents hitting model limits (e.g., >2000 tokens)
- Message threshold: Prevents conversation from getting too long (e.g., >10 messages)
- Keep recent: Preserves the most relevant context (e.g., last 4 messages)


In [14]:
def should_summarize(
    messages: List[ConversationMessage],
    token_threshold: int = 2000,
    message_threshold: int = 10,
    keep_recent: int = 4
) -> bool:
    """
    Determine if conversation needs summarization.

    Args:
        messages: List of conversation messages
        token_threshold: Summarize when total tokens exceed this
        message_threshold: Summarize when message count exceeds this
        keep_recent: Number of recent messages to keep unsummarized

    Returns:
        True if summarization is needed
    """
    # Don't summarize if we have very few messages
    if len(messages) <= keep_recent:
        return False

    # Calculate total tokens
    total_tokens = sum(msg.token_count for msg in messages)

    # Summarize if either threshold is exceeded
    return (total_tokens > token_threshold or
            len(messages) > message_threshold)

print("✅ should_summarize() function defined")


✅ should_summarize() function defined


#### Step 3: Create a prompt template for summarization

**What we're building:** A carefully crafted prompt that instructs the LLM on how to summarize conversations.

**Why it's needed:** Generic summarization loses important details. We need domain-specific instructions that preserve what matters for course advisory conversations.

**How it works:**
- Specifies the context (student-advisor conversation)
- Lists exactly what to preserve (decisions, requirements, goals, courses, issues)
- Requests structured output (bullet points for clarity)
- Emphasizes being "specific and actionable" (not vague summaries)

**Design principle:** The prompt template is the "instructions" for the summarization LLM. Better instructions = better summaries.


In [15]:
summarization_prompt_template = """You are summarizing a conversation between a student and a course advisor.

Create a concise summary that preserves:
1. Key decisions made
2. Important requirements or prerequisites discussed
3. Student's goals, preferences, and constraints
4. Specific courses mentioned and recommendations given
5. Any problems or issues that need follow-up

Format as bullet points. Be specific and actionable.

Conversation to summarize:
{conversation}

Summary:"""

print("✅ Summarization prompt template defined")


✅ Summarization prompt template defined


#### Step 4: Create a function to generate summaries using the LLM

**What we're building:** A function that takes messages and produces an intelligent summary using an LLM.

**Why it's needed:** This is where the actual summarization happens. We need to:
- Format the conversation for the LLM
- Call the LLM with our prompt template
- Package the summary as a system message

**How it works:**
1. Formats messages as "User: ..." and "Assistant: ..." text
2. Inserts formatted conversation into the prompt template
3. Calls the LLM asynchronously (non-blocking)
4. Wraps the summary in `[CONVERSATION SUMMARY]` marker for easy identification
5. Returns as a system message (distinguishes it from user/assistant messages)

**Why async?** Summarization can take 1-3 seconds. Async allows other operations to continue while waiting for the LLM response.


In [16]:
async def create_summary(
    messages: List[ConversationMessage],
    llm: ChatOpenAI
) -> ConversationMessage:
    """
    Create intelligent summary of conversation messages.

    Args:
        messages: List of messages to summarize
        llm: Language model for generating summary

    Returns:
        ConversationMessage containing the summary
    """
    # Format conversation for summarization
    conversation_text = "\n".join([
        f"{msg.role.title()}: {msg.content}"
        for msg in messages
    ])

    # Generate summary using LLM
    prompt = summarization_prompt_template.format(conversation=conversation_text)
    response = await llm.ainvoke([HumanMessage(content=prompt)])

    summary_content = f"[CONVERSATION SUMMARY]\n{response.content}"

    # Create summary message
    summary_msg = ConversationMessage(
        role="system",
        content=summary_content,
        timestamp=messages[-1].timestamp
    )

    return summary_msg

print("✅ create_summary() function defined")


✅ create_summary() function defined


#### Step 5: Create a function to compress conversations

**What we're building:** The main compression function that orchestrates the entire summarization process.

**Why it's needed:** This ties together all the previous components into a single, easy-to-use function that:
- Decides whether to summarize
- Splits messages into old vs. recent
- Generates the summary
- Returns the compressed conversation

**How it works:**
1. **Check:** Calls `should_summarize()` to see if compression is needed
2. **Split:** Divides messages into `old_messages` (to summarize) and `recent_messages` (to keep)
3. **Summarize:** Calls `create_summary()` on old messages
4. **Combine:** Returns `[summary] + recent_messages`

**The result:** A conversation that's 50-80% smaller but preserves all essential information.

**Example:**
- Input: 20 messages (4,000 tokens)
- Output: 1 summary + 4 recent messages (1,200 tokens)
- Savings: 70% reduction in tokens


In [17]:
async def compress_conversation(
    messages: List[ConversationMessage],
    llm: ChatOpenAI,
    token_threshold: int = 2000,
    message_threshold: int = 10,
    keep_recent: int = 4
) -> List[ConversationMessage]:
    """
    Compress conversation by summarizing old messages and keeping recent ones.

    Args:
        messages: List of conversation messages
        llm: Language model for generating summaries
        token_threshold: Summarize when total tokens exceed this
        message_threshold: Summarize when message count exceeds this
        keep_recent: Number of recent messages to keep unsummarized

    Returns:
        List of messages: [summary] + [recent messages]
    """
    # Check if summarization is needed
    if not should_summarize(messages, token_threshold, message_threshold, keep_recent):
        return messages

    # Split into old and recent
    old_messages = messages[:-keep_recent]
    recent_messages = messages[-keep_recent:]

    if not old_messages:
        return messages

    # Summarize old messages
    summary = await create_summary(old_messages, llm)

    # Return summary + recent messages
    return [summary] + recent_messages

print("✅ compress_conversation() function defined")


✅ compress_conversation() function defined


#### Step 6: Combine into a reusable class

Now that we've built and tested each component, let's combine them into a reusable class.


In [18]:
class ConversationSummarizer:
    """Manages conversation summarization to keep token counts manageable."""

    def __init__(
        self,
        llm: ChatOpenAI,
        token_threshold: int = 2000,
        message_threshold: int = 10,
        keep_recent: int = 4
    ):
        """
        Initialize the summarizer.

        Args:
            llm: Language model for generating summaries
            token_threshold: Summarize when total tokens exceed this
            message_threshold: Summarize when message count exceeds this
            keep_recent: Number of recent messages to keep unsummarized
        """
        self.llm = llm
        self.token_threshold = token_threshold
        self.message_threshold = message_threshold
        self.keep_recent = keep_recent
        self.summarization_prompt = summarization_prompt_template

    def should_summarize(self, messages: List[ConversationMessage]) -> bool:
        """Determine if conversation needs summarization."""
        return should_summarize(
            messages,
            self.token_threshold,
            self.message_threshold,
            self.keep_recent
        )

    async def summarize_conversation(
        self,
        messages: List[ConversationMessage]
    ) -> ConversationMessage:
        """Create intelligent summary of conversation messages."""
        return await create_summary(messages, self.llm)

    async def compress_conversation(
        self,
        messages: List[ConversationMessage]
    ) -> List[ConversationMessage]:
        """Compress conversation by summarizing old messages and keeping recent ones."""
        return await compress_conversation(
            messages,
            self.llm,
            self.token_threshold,
            self.message_threshold,
            self.keep_recent
        )

print("✅ ConversationSummarizer class defined")


✅ ConversationSummarizer class defined


### Demo 3: Test Summarization

Let's test the summarizer with a sample conversation.

#### Step 1: Create a sample conversation


In [19]:
# Create a sample long conversation
sample_conversation = [
    ConversationMessage("user", "Hi, I'm interested in learning about machine learning courses"),
    ConversationMessage("assistant", "Great! Redis University offers several ML courses. CS401 Machine Learning is our flagship course. It covers supervised learning, neural networks, and practical applications."),
    ConversationMessage("user", "What are the prerequisites for CS401?"),
    ConversationMessage("assistant", "CS401 requires CS201 Data Structures and MATH301 Linear Algebra. Have you completed these courses?"),
    ConversationMessage("user", "I've completed CS101 but not CS201 yet"),
    ConversationMessage("assistant", "Perfect! CS201 is the next logical step. It covers algorithms and data structures essential for ML. It's offered every semester."),
    ConversationMessage("user", "How difficult is MATH301?"),
    ConversationMessage("assistant", "MATH301 is moderately challenging. It covers vectors, matrices, and eigenvalues used in ML algorithms. Most students find it manageable with consistent practice."),
    ConversationMessage("user", "Can I take both CS201 and MATH301 together?"),
    ConversationMessage("assistant", "Yes, that's a good combination! They complement each other well. Many students take them concurrently."),
    ConversationMessage("user", "What about CS401 after that?"),
    ConversationMessage("assistant", "CS401 is perfect after completing both prerequisites. It's our most popular AI course with hands-on projects."),
    ConversationMessage("user", "When is CS401 offered?"),
    ConversationMessage("assistant", "CS401 is offered in Fall and Spring semesters. The Fall section typically fills up quickly, so register early!"),
    ConversationMessage("user", "Great! What's the workload like?"),
    ConversationMessage("assistant", "CS401 requires about 10-12 hours per week including lectures, assignments, and projects. There are 4 major projects throughout the semester."),
]

# Calculate original metrics
original_token_count = sum(msg.token_count for msg in sample_conversation)
print(f"Original conversation:")
print(f"  Messages: {len(sample_conversation)}")
print(f"  Total tokens: {original_token_count}")
print(f"  Average tokens per message: {original_token_count / len(sample_conversation):.1f}")


Original conversation:
  Messages: 16
  Total tokens: 261
  Average tokens per message: 16.3


#### Step 2: Configure the summarizer


In [20]:
# Test summarization
summarizer = ConversationSummarizer(
    llm=llm,
    token_threshold=500,  # Low threshold for demo
    message_threshold=10,
    keep_recent=4
)

print(f"Summarizer configuration:")
print(f"  Token threshold: {summarizer.token_threshold}")
print(f"  Message threshold: {summarizer.message_threshold}")
print(f"  Keep recent: {summarizer.keep_recent}")


Summarizer configuration:
  Token threshold: 500
  Message threshold: 10
  Keep recent: 4


#### Step 3: Check if summarization is needed


In [21]:
# Check if summarization is needed
should_summarize_result = summarizer.should_summarize(sample_conversation)
print(f"Should summarize? {should_summarize_result}")


Should summarize? True


#### Step 4: Compress the conversation


In [22]:
# Compress the conversation
compressed = await summarizer.compress_conversation(sample_conversation)

compressed_token_count = sum(msg.token_count for msg in compressed)
token_savings = original_token_count - compressed_token_count
savings_percentage = (token_savings / original_token_count) * 100

print(f"After summarization:")
print(f"  Messages: {len(compressed)}")
print(f"  Total tokens: {compressed_token_count}")
print(f"  Token savings: {token_savings} ({savings_percentage:.1f}%)")


After summarization:
  Messages: 5
  Total tokens: 294
  Token savings: -33 (-12.6%)


#### Step 5: Examine the compressed conversation structure


In [23]:
print("Compressed conversation structure:")
for i, msg in enumerate(compressed):
    role_icon = "📋" if msg.role == "system" else "👤" if msg.role == "user" else "🤖"
    content_preview = msg.content[:80].replace('\n', ' ')
    print(f"  {i+1}. {role_icon} [{msg.role}] {content_preview}...")
    print(f"     Tokens: {msg.token_count}")


Compressed conversation structure:
  1. 📋 [system] [CONVERSATION SUMMARY] - **Key Decisions Made:**   - The student decided to take...
     Tokens: 230
  2. 👤 [user] When is CS401 offered?...
     Tokens: 6
  3. 🤖 [assistant] CS401 is offered in Fall and Spring semesters. The Fall section typically fills ...
     Tokens: 22
  4. 👤 [user] Great! What's the workload like?...
     Tokens: 7
  5. 🤖 [assistant] CS401 requires about 10-12 hours per week including lectures, assignments, and p...
     Tokens: 29


#### Results Analysis

**What happened:**
- Original: 16 messages with ~{original_token_count} tokens
- Compressed: {len(compressed)} messages (1 summary + 4 recent)
- Savings: ~{savings_percentage:.0f}% token reduction

**Key benefits:**
- Preserved recent context (last 4 messages)
- Summarized older messages into key facts
- Maintained conversation continuity
- Reduced token costs significantly


---

## 🔧 Part 3: Summarization is not the only strategy, there are other Context Compression Strategies

Summarization compresses conversation history by generating condensed representations of past messages. However, it's not the only viable approach to context management, and it's not always optimal.

### Why Not Always Optimal?

Summarization is powerful but introduces trade-offs that make it suboptimal for certain scenarios.

**Technical Trade-offs:**

1. **Latency Overhead**
   - Each summarization requires an LLM API call
   - Adds 1-3 seconds per compression (vs. <10ms for truncation)
   - Blocks conversation flow in real-time applications

2. **Cost Multiplication**
   - Input tokens: Entire conversation to summarize
   - Output tokens: Generated summary
   - At scale: 1,000 conversations/day = 1,000+ extra LLM calls

3. **Lossy Compression**
   - Summaries paraphrase, don't preserve exact wording
   - Loses temporal sequence and conversation flow
   - Can't reconstruct original messages
   - Unacceptable for legal, medical, or compliance contexts

4. **Implementation Complexity**
   - Requires async operations and error handling
   - Needs domain-specific prompt engineering
   - Unpredictable compression ratios
   - Summary quality varies with prompt design

**When to Use Alternatives:**

| Scenario | Better Strategy | Why |
|----------|----------------|-----|
| Short conversations (<5 turns) | None (keep all) | Overhead exceeds benefit |
| Real-time chat | Truncation | Zero latency |
| Cost-sensitive (high volume) | Priority-based | No API calls |
| Verbatim accuracy required | Truncation | Preserves exact wording |
| Balanced quality + speed | Priority-based | Intelligent selection, no LLM |

**Decision Framework:**
- **Speed-critical** → Truncation (instant, predictable)
- **Cost-sensitive** → Priority-based (no API calls, intelligent)
- **Quality-critical** → Summarization (preserves meaning, expensive)
- **Hybrid** → Truncation + summarization (fast for most, summarize when needed)

### Theory: Four Compression Approaches

Let's explore four different strategies, each with different trade-offs:

**1. Truncation (Token-Aware)**
- Keep recent messages within token budget
- ✅ Pros: Fast, no LLM calls, respects context limits
- ❌ Cons: Variable message count, loses old context
- **Best for:** Token-constrained applications, API limits

**2. Sliding Window (Message-Aware)**
- Keep exactly N most recent messages
- ✅ Pros: Fastest, predictable count, constant memory
- ❌ Cons: May exceed token limits, loses old context
- **Best for:** Fixed-size buffers, real-time chat

**3. Priority-Based (Balanced)**
- Score messages by importance, keep highest-scoring
- ✅ Pros: Preserves important context, no LLM calls
- ❌ Cons: Requires good scoring logic, may lose temporal flow
- **Best for:** Production applications needing balance

**4. Summarization (High Quality)**
- Use LLM to create intelligent summaries
- ✅ Pros: Preserves meaning, high quality
- ❌ Cons: Slower, costs tokens, requires LLM call
- **Best for:** High-value conversations, quality-critical applications


### Building Compression Strategies Step-by-Step

Let's build each strategy incrementally, starting with the simplest.

#### Step 1: Define a base interface for compression strategies


In [ ]:
class CompressionStrategy:
    """Base class for compression strategies."""

    def compress(
        self,
        messages: List[ConversationMessage],
        max_tokens: int
    ) -> List[ConversationMessage]:
        """Compress messages to fit within max_tokens."""
        raise NotImplementedError

print("✅ CompressionStrategy base class defined")


#### Step 2: Implement Truncation Strategy (Simplest)

This strategy simply keeps the most recent messages that fit within the token budget.


In [ ]:
class TruncationStrategy(CompressionStrategy):
    """Keep only the most recent messages within token budget."""

    def compress(
        self,
        messages: List[ConversationMessage],
        max_tokens: int
    ) -> List[ConversationMessage]:
        """Keep most recent messages within token budget."""
        compressed = []
        total_tokens = 0

        # Work backwards from most recent
        for msg in reversed(messages):
            if total_tokens + msg.token_count <= max_tokens:
                compressed.insert(0, msg)
                total_tokens += msg.token_count
            else:
                break

        return compressed

print("✅ TruncationStrategy implemented")

# Test it
truncation = TruncationStrategy()
test_result = truncation.compress(sample_conversation, max_tokens=500)
print(f"   Truncation test: {len(sample_conversation)} messages → {len(test_result)} messages")


#### Step 2.5: Implement Sliding Window Strategy (Simplest)

**What we're building:** A strategy that maintains a fixed-size window of the N most recent messages.

**Why it's different from truncation:**
- **Truncation:** Reactive - keeps messages until token budget exceeded, then removes oldest
- **Sliding Window:** Proactive - always maintains exactly N messages regardless of tokens

**When to use:**
- Real-time chat where you want constant context size
- Systems with predictable message patterns
- When simplicity matters more than token optimization

**Trade-off:** May exceed token limits if messages are very long.

**How it works:** Simply returns the last N messages using Python list slicing (`messages[-N:]`).


In [ ]:
class SlidingWindowStrategy(CompressionStrategy):
    """Keep only the last N messages (fixed window size)."""

    def __init__(self, window_size: int = 10):
        """
        Initialize sliding window strategy.

        Args:
            window_size: Number of recent messages to keep
        """
        self.window_size = window_size

    def compress(
        self,
        messages: List[ConversationMessage],
        max_tokens: int
    ) -> List[ConversationMessage]:
        """
        Keep only the last N messages.

        Note: Ignores max_tokens parameter - always keeps exactly window_size messages.
        """
        if len(messages) <= self.window_size:
            return messages

        return messages[-self.window_size:]

print("✅ SlidingWindowStrategy implemented")

# Test it
sliding_window = SlidingWindowStrategy(window_size=6)
test_result = sliding_window.compress(sample_conversation, max_tokens=500)
test_tokens = sum(msg.token_count for msg in test_result)

print(f"   Sliding window test: {len(sample_conversation)} messages → {len(test_result)} messages")
print(f"   Token count: {test_tokens} tokens (budget was {500})")


**Analysis:**

The sliding window strategy demonstrates:
- ✅ **Predictable output:** Always returns exactly 6 messages (or fewer if conversation is shorter)
- ✅ **O(1) complexity:** Just slices the list - fastest possible implementation
- ⚠️ **Token-agnostic:** Returned {test_tokens} tokens, which may or may not fit the 500 token budget
- ✅ **Simplest code:** One line implementation (`messages[-N:]`)

**Key insight:** Sliding window prioritizes **predictability** over **token optimization**. Use it when you need constant context size and can tolerate variable token counts.

**Comparison with Truncation:**
- **Truncation:** "Keep as many recent messages as fit in budget" → Variable count, guaranteed under limit
- **Sliding Window:** "Keep exactly N recent messages" → Fixed count, may exceed limit


#### Step 3: Implement Priority-Based Strategy (Intelligent Selection)

This strategy scores messages by importance and keeps the highest-scoring ones.

First, let's create a function to calculate message importance:


In [ ]:
def calculate_message_importance(msg: ConversationMessage) -> float:
    """
    Calculate importance score for a message.

    Higher scores = more important.
    """
    score = 0.0
    content_lower = msg.content.lower()

    # Course codes are important (CS401, MATH301, etc.)
    if any(code in content_lower for code in ['cs', 'math', 'eng']):
        score += 2.0

    # Questions are important
    if '?' in msg.content:
        score += 1.5

    # Prerequisites and requirements are important
    if any(word in content_lower for word in ['prerequisite', 'require', 'need']):
        score += 1.5

    # Preferences and goals are important
    if any(word in content_lower for word in ['prefer', 'want', 'goal', 'interested']):
        score += 1.0

    # User messages slightly more important (their needs)
    if msg.role == 'user':
        score += 0.5

    # Longer messages often have more content
    if msg.token_count > 50:
        score += 0.5

    return score

print("✅ calculate_message_importance() function defined")

# Test it
test_scores = [(msg.content[:50], calculate_message_importance(msg))
               for msg in sample_conversation[:3]]
print("\nExample importance scores:")
for content, score in test_scores:
    print(f"  Score {score:.1f}: {content}...")


Now let's create the Priority-Based strategy class:


In [ ]:
class PriorityBasedStrategy(CompressionStrategy):
    """Keep highest-priority messages within token budget."""

    def calculate_importance(self, msg: ConversationMessage) -> float:
        """Calculate importance score for a message."""
        return calculate_message_importance(msg)

    def compress(
        self,
        messages: List[ConversationMessage],
        max_tokens: int
    ) -> List[ConversationMessage]:
        """Keep highest-priority messages within token budget."""
        # Score each message
        scored_messages = [
            (self.calculate_importance(msg), i, msg)
            for i, msg in enumerate(messages)
        ]

        # Sort by score (descending), then by index to maintain some order
        scored_messages.sort(key=lambda x: (-x[0], x[1]))

        # Select messages within budget
        selected = []
        total_tokens = 0

        for score, idx, msg in scored_messages:
            if total_tokens + msg.token_count <= max_tokens:
                selected.append((idx, msg))
                total_tokens += msg.token_count

        # Sort by original index to maintain conversation flow
        selected.sort(key=lambda x: x[0])

        return [msg for idx, msg in selected]

print("✅ PriorityBasedStrategy implemented")

# Test it
priority = PriorityBasedStrategy()
test_result = priority.compress(sample_conversation, max_tokens=800)
print(f"   Priority-based test: {len(sample_conversation)} messages → {len(test_result)} messages")


#### Step 4: Implement Summarization Strategy (Highest Quality)

This strategy uses our ConversationSummarizer to create intelligent summaries.


In [ ]:
class SummarizationStrategy(CompressionStrategy):
    """Use LLM to create intelligent summaries."""

    def __init__(self, summarizer: ConversationSummarizer):
        self.summarizer = summarizer

    async def compress_async(
        self,
        messages: List[ConversationMessage],
        max_tokens: int
    ) -> List[ConversationMessage]:
        """Compress using summarization (async)."""
        # Use the summarizer's logic
        return await self.summarizer.compress_conversation(messages)

    def compress(
        self,
        messages: List[ConversationMessage],
        max_tokens: int
    ) -> List[ConversationMessage]:
        """Synchronous wrapper (not recommended, use compress_async)."""
        raise NotImplementedError("Use compress_async for summarization strategy")

print("✅ SummarizationStrategy implemented")


### Demo 4: Compare Compression Strategies

Let's compare all four strategies on the same conversation to understand their trade-offs.

#### Step 1: Set up the test


In [ ]:
# Use the same sample conversation from before
test_conversation = sample_conversation.copy()
max_tokens = 800  # Target token budget

original_tokens = sum(msg.token_count for msg in test_conversation)
print(f"Original conversation: {len(test_conversation)} messages, {original_tokens} tokens")
print(f"Target budget: {max_tokens} tokens\n")


#### Step 2: Test Truncation Strategy


In [ ]:
truncation = TruncationStrategy()
truncated = truncation.compress(test_conversation, max_tokens)
truncated_tokens = sum(msg.token_count for msg in truncated)

print(f"TRUNCATION STRATEGY")
print(f"  Result: {len(truncated)} messages, {truncated_tokens} tokens")
print(f"  Savings: {original_tokens - truncated_tokens} tokens")
print(f"  Kept messages: {[i for i, msg in enumerate(test_conversation) if msg in truncated]}")


#### Step 2.5: Test Sliding Window Strategy


In [ ]:
sliding_window = SlidingWindowStrategy(window_size=6)
windowed = sliding_window.compress(test_conversation, max_tokens)
windowed_tokens = sum(msg.token_count for msg in windowed)

print(f"SLIDING WINDOW STRATEGY")
print(f"  Result: {len(windowed)} messages, {windowed_tokens} tokens")
print(f"  Savings: {original_tokens - windowed_tokens} tokens")
print(f"  Kept messages: {[i for i, msg in enumerate(test_conversation) if msg in windowed]}")
print(f"  Token budget: {windowed_tokens}/{max_tokens} ({'within' if windowed_tokens <= max_tokens else 'EXCEEDS'} limit)")


**Analysis:**

The sliding window kept:
- **Exactly 6 messages** (last 6 from the conversation)
- **Most recent context only** (indices show the final messages)
- **{windowed_tokens} tokens** (may or may not fit budget)

**Key difference from truncation:**
- **Truncation:** Kept {len(truncated)} messages to stay under {max_tokens} tokens
- **Sliding Window:** Kept exactly 6 messages, resulting in {windowed_tokens} tokens

**Behavior pattern:**
- Truncation: "Fill the budget" → Variable count, guaranteed fit
- Sliding Window: "Fixed window" → Constant count, may exceed budget


#### Step 3: Test Priority-Based Strategy


In [ ]:
priority = PriorityBasedStrategy()
prioritized = priority.compress(test_conversation, max_tokens)
prioritized_tokens = sum(msg.token_count for msg in prioritized)

print(f"PRIORITY-BASED STRATEGY")
print(f"  Result: {len(prioritized)} messages, {prioritized_tokens} tokens")
print(f"  Savings: {original_tokens - prioritized_tokens} tokens")
print(f"  Kept messages: {[i for i, msg in enumerate(test_conversation) if msg in prioritized]}")


Let's examine which messages were selected and why:


In [ ]:
# Show importance scores for selected messages
print("Sample importance scores:")
for i in [0, 2, 4, 6]:
    if i < len(test_conversation):
        score = priority.calculate_importance(test_conversation[i])
        preview = test_conversation[i].content[:50]
        print(f"  Message {i}: {score:.1f} - \"{preview}...\"")


#### Step 4: Test Summarization Strategy


In [ ]:
summarization = SummarizationStrategy(summarizer)
summarized = await summarization.compress_async(test_conversation, max_tokens)
summarized_tokens = sum(msg.token_count for msg in summarized)

print(f"SUMMARIZATION STRATEGY")
print(f"  Result: {len(summarized)} messages, {summarized_tokens} tokens")
print(f"  Savings: {original_tokens - summarized_tokens} tokens")
print(f"  Structure: 1 summary + {len(summarized) - 1} recent messages")


#### Step 5: Compare all strategies


In [ ]:
print("COMPARISON SUMMARY")
print("=" * 80)
print(f"{'Strategy':<20} {'Messages':<12} {'Tokens':<12} {'Savings':<12} {'Quality'}")
print("-" * 80)

strategies = [
    ("Original", len(test_conversation), original_tokens, 0, "N/A"),
    ("Truncation", len(truncated), truncated_tokens, original_tokens - truncated_tokens, "Low"),
    ("Sliding Window", len(windowed), windowed_tokens, original_tokens - windowed_tokens, "Low"),
    ("Priority-Based", len(prioritized), prioritized_tokens, original_tokens - prioritized_tokens, "Medium"),
    ("Summarization", len(summarized), summarized_tokens, original_tokens - summarized_tokens, "High"),
]

for name, msgs, tokens, savings, quality in strategies:
    savings_pct = f"({savings/original_tokens*100:.0f}%)" if savings > 0 else ""
    print(f"{name:<20} {msgs:<12} {tokens:<12} {savings:<5} {savings_pct:<6} {quality}")


#### Key Takeaways

**Truncation (Token-Aware):**
- Keeps messages within token budget
- Variable message count, guaranteed under limit
- Good for: API token limits, cost control

**Sliding Window (Message-Aware):**
- Keeps exactly N most recent messages
- Fixed message count, may exceed token budget
- Good for: Real-time chat, predictable context size

**Priority-Based (Intelligent):**
- Scores and keeps important messages
- Preserves key information across conversation
- Good for: Most production applications, balanced approach

**Summarization (Highest Quality):**
- Uses LLM to preserve meaning
- Highest quality, but requires API call (cost + latency)
- Good for: High-value conversations, support tickets, advisory sessions

**Decision Framework:**
- **Speed-critical** → Truncation or Sliding Window (instant, no LLM)
- **Cost-sensitive** → Priority-Based (intelligent, no API calls)
- **Quality-critical** → Summarization (preserves meaning, expensive)
- **Predictable context** → Sliding Window (constant message count)


---

## 🔄 Part 4: Agent Memory Server Integration

The Agent Memory Server provides automatic summarization. Let's configure and test it.


### 🔬 Research Foundation: Hierarchical Memory Management

Packer et al. (2023) in ["MemGPT: Towards LLMs as Operating Systems"](https://arxiv.org/abs/2310.08560) introduced a groundbreaking approach to memory management:

**Key Insight:** Treat LLM context like an operating system's memory hierarchy:
- **Main Context** (like RAM): Limited, fast access
- **External Memory** (like disk): Unlimited, slower access
- **Intelligent Paging**: Move data between tiers based on relevance

**Their Virtual Context Management System:**
1. Fixed-size main context (within token limits)
2. Recursive memory retrieval from external storage
3. LLM decides what to page in/out based on task needs

**Practical Implications:**
- Hierarchical approach enables unbounded conversations
- Intelligent data movement between memory tiers
- Transparent to application code

**This is exactly what Agent Memory Server implements:**
- **Working Memory** (Main Context): Session-scoped conversation messages
- **Long-term Memory** (External Memory): Persistent facts, preferences, goals
- **Automatic Management**: Extracts important information from working → long-term


### 🔬 Research-Backed Implementation

The Agent Memory Server implements the research findings we've discussed:

**From "Lost in the Middle" (Liu et al., 2023):**
- Keeps recent messages at the end of context (optimal position)
- Summarizes middle content to avoid performance degradation
- Maintains fixed context size for consistent performance

**From "Recursive Summarization" (Wang et al., 2023):**
- Automatically creates summaries when thresholds are exceeded
- Preserves key information across long conversations
- Enables unbounded conversation length

**From "MemGPT" (Packer et al., 2023):**
- Hierarchical memory management (working + long-term)
- Intelligent data movement between memory tiers
- Transparent to application code

**Production Best Practices** (Anthropic, Vellum AI):
- Configurable thresholds for different use cases
- Multiple strategies (truncation, summarization, hybrid)
- Scalable and production-ready architecture

**References:**
- Packer, C., Wooders, S., Lin, K., et al. (2023). MemGPT: Towards LLMs as Operating Systems. *arXiv preprint arXiv:2310.08560*.
- Vellum AI. (2024). [How Should I Manage Memory for my LLM Chatbot?](https://www.vellum.ai/blog/how-should-i-manage-memory-for-my-llm-chatbot)
- Anthropic. (2024). [Effective Context Engineering for AI Agents](https://www.anthropic.com/engineering/effective-context-engineering-for-ai-agents)


### Theory: Automatic Memory Management

**Agent Memory Server Features:**
- ✅ Automatic summarization when thresholds are exceeded
- ✅ Configurable strategies (recent + summary, sliding window, full summary)
- ✅ Transparent to your application code
- ✅ Production-ready and scalable

**How It Works:**
1. You add messages to working memory normally
2. Server monitors message count and token count
3. When threshold is exceeded, server automatically summarizes
4. Old messages are replaced with summary
5. Recent messages are kept for context
6. Your application retrieves the compressed memory

**Configuration Options:**
- `message_threshold`: Summarize after N messages (default: 20)
- `token_threshold`: Summarize after N tokens (default: 4000)
- `keep_recent`: Number of recent messages to keep (default: 4)
- `strategy`: "recent_plus_summary", "sliding_window", or "full_summary"


### Demo 5: Test Automatic Summarization

Let's test the Agent Memory Server's automatic summarization with a long conversation.

#### Step 1: Create a test session


In [ ]:
# Create a test session
test_session_id = f"long_conversation_test_{int(time.time())}"
test_student_id = "student_memory_test"

print(f"Testing automatic summarization")
print(f"Session ID: {test_session_id}")
print(f"Student ID: {test_student_id}")


#### Step 2: Define a long conversation (25 turns = 50 messages)


In [ ]:
conversation_turns = [
    ("I'm interested in machine learning", "Great! CS401 Machine Learning is perfect for you."),
    ("What are the prerequisites?", "You'll need CS201 Data Structures and MATH301 Linear Algebra."),
    ("I've completed CS101", "Perfect! CS201 is your next step."),
    ("How difficult is CS201?", "It's moderately challenging but very rewarding."),
    ("When is it offered?", "CS201 is offered every semester - Fall, Spring, and Summer."),
    ("What about MATH301?", "MATH301 covers linear algebra essentials for ML."),
    ("Can I take both together?", "Yes, many students take CS201 and MATH301 concurrently."),
    ("How long will it take?", "If you take both, you can start CS401 in about 4-6 months."),
    ("What's the workload?", "Expect 10-12 hours per week for each course."),
    ("Are there online options?", "Yes, both courses have online and in-person sections."),
    ("Which format is better?", "Online offers flexibility, in-person offers more interaction."),
    ("What about CS401 after that?", "CS401 is our flagship ML course with hands-on projects."),
    ("How many projects?", "CS401 has 4 major projects throughout the semester."),
    ("What topics are covered?", "Supervised learning, neural networks, deep learning, and NLP."),
    ("Is there a final exam?", "Yes, there's a comprehensive final exam worth 30% of your grade."),
    ("What's the pass rate?", "About 85% of students pass CS401 on their first attempt."),
    ("Are there TAs available?", "Yes, we have 3 TAs for CS401 with office hours daily."),
    ("What programming language?", "CS401 uses Python with TensorFlow and PyTorch."),
    ("Do I need a GPU?", "Recommended but not required. We provide cloud GPU access."),
    ("What's the class size?", "CS401 typically has 30-40 students per section."),
    ("Can I audit the course?", "Yes, auditing is available but you won't get credit."),
    ("What's the cost?", "CS401 is $1,200 for credit, $300 for audit."),
    ("Are there scholarships?", "Yes, we offer merit-based scholarships. Apply early!"),
    ("When should I apply?", "Applications open 2 months before each semester starts."),
    ("Thanks for the help!", "You're welcome! Feel free to reach out with more questions."),
]

print(f"Prepared {len(conversation_turns)} conversation turns ({len(conversation_turns)*2} messages)")


#### Step 3: Add messages to working memory

The Agent Memory Server will automatically monitor and summarize when thresholds are exceeded.


In [ ]:
# Get or create working memory
_, working_memory = await memory_client.get_or_create_working_memory(
    session_id=test_session_id,
    user_id=test_student_id,
    model_name="gpt-4o"
)

print("Adding messages to working memory...")
print("=" * 80)

for i, (user_msg, assistant_msg) in enumerate(conversation_turns, 1):
    # Add messages to working memory
    working_memory.messages.extend([
        MemoryMessage(role="user", content=user_msg),
        MemoryMessage(role="assistant", content=assistant_msg)
    ])

    # Save to Memory Server
    await memory_client.put_working_memory(
        session_id=test_session_id,
        memory=working_memory,
        user_id=test_student_id,
        model_name="gpt-4o"
    )

    # Show progress every 5 turns
    if i % 5 == 0:
        print(f"Turn {i:2d}: Added messages (total: {i*2} messages)")

print(f"\n✅ Added {len(conversation_turns)} turns ({len(conversation_turns)*2} messages)")


#### Step 4: Retrieve working memory and check for summarization


In [ ]:
# Retrieve the latest working memory
_, working_memory = await memory_client.get_or_create_working_memory(
    session_id=test_session_id,
    user_id=test_student_id,
    model_name="gpt-4o"
)

print("Working Memory Status:")
print(f"  Messages in memory: {len(working_memory.messages)}")
print(f"  Original messages added: {len(conversation_turns)*2}")


#### Step 5: Analyze the results

**What we're checking:** Did the Agent Memory Server automatically detect the threshold and trigger summarization?

**Why this matters:** Automatic summarization means you don't have to manually manage memory - the system handles it transparently.


In [ ]:
if len(working_memory.messages) < len(conversation_turns)*2:
    print("\n✅ Automatic summarization occurred!")
    print(f"  Compression: {len(conversation_turns)*2} → {len(working_memory.messages)} messages")

    # Calculate compression ratio
    compression_ratio = len(working_memory.messages) / (len(conversation_turns)*2)
    print(f"  Compression ratio: {compression_ratio:.2f}x (kept {compression_ratio*100:.0f}% of messages)")

    # Check for summary message
    summary_messages = [msg for msg in working_memory.messages if '[SUMMARY]' in msg.content or msg.role == 'system']
    if summary_messages:
        print(f"  Summary messages found: {len(summary_messages)}")
        print(f"\n  Summary preview:")
        for msg in summary_messages[:1]:  # Show first summary
            content_preview = msg.content[:200].replace('\n', ' ')
            print(f"  {content_preview}...")

        # Analyze what was preserved
        recent_messages = [msg for msg in working_memory.messages if msg.role in ['user', 'assistant']]
        print(f"\n  Recent messages preserved: {len(recent_messages)}")
        print(f"  Strategy: Summary + recent messages (optimal for 'Lost in the Middle')")
else:
    print("\nℹ️  No summarization yet (threshold not reached)")
    print(f"  Current: {len(working_memory.messages)} messages")
    print(f"  Threshold: 20 messages or 4000 tokens")


#### Step 6: Calculate token savings and analyze efficiency

**What we're measuring:** The economic and performance impact of summarization.

**Why this matters:**
- **Cost savings:** Fewer tokens = lower API costs
- **Performance:** Smaller context = faster responses
- **Quality:** Compressed context avoids "Lost in the Middle" problem


In [ ]:
original_tokens = sum(count_tokens(user_msg) + count_tokens(assistant_msg) for user_msg, assistant_msg in conversation_turns)
current_tokens = sum(count_tokens(msg.content) for msg in working_memory.messages)

print("=" * 80)
print("TOKEN EFFICIENCY ANALYSIS")
print("=" * 80)

print(f"\n📊 Token Counts:")
print(f"  Original tokens: {original_tokens:,}")
print(f"  Current tokens: {current_tokens:,}")

if current_tokens < original_tokens:
    savings = original_tokens - current_tokens
    savings_pct = (savings / original_tokens) * 100

    print(f"\n💰 Savings:")
    print(f"  Token savings: {savings:,} tokens ({savings_pct:.1f}%)")

    # Calculate cost savings (GPT-4o pricing: $0.0025 per 1K input tokens)
    cost_per_1k = 0.0025
    original_cost = (original_tokens / 1000) * cost_per_1k
    current_cost = (current_tokens / 1000) * cost_per_1k
    cost_savings = original_cost - current_cost

    print(f"  Cost per query: ${original_cost:.4f} → ${current_cost:.4f}")
    print(f"  Cost savings: ${cost_savings:.4f} per query")

    # Extrapolate to scale
    queries_per_day = 1000
    daily_savings = cost_savings * queries_per_day
    monthly_savings = daily_savings * 30

    print(f"\n📈 At Scale (1,000 queries/day):")
    print(f"  Daily savings: ${daily_savings:.2f}")
    print(f"  Monthly savings: ${monthly_savings:.2f}")
    print(f"  Annual savings: ${monthly_savings * 12:.2f}")

    print(f"\n⚡ Performance Benefits:")
    print(f"  Reduced latency: ~{savings_pct * 0.3:.0f}% faster (fewer tokens to process)")
    print(f"  Better quality: Recent context at optimal position (end of context)")
    print(f"  Avoids 'Lost in the Middle': Summary at beginning, recent at end")

    print(f"\n✅ Automatic memory management is working efficiently!")
else:
    print(f"\nℹ️  No compression yet (within thresholds)")
    print(f"  Waiting for: >20 messages OR >4000 tokens")


---

## 🎯 Part 5: Decision Framework

How do you choose which compression strategy to use? Let's build a decision framework.


### 🔬 Synthesizing Research into Practice

Our decision framework synthesizes findings from all the research we've discussed:

**From "Lost in the Middle" (Liu et al., 2023):**
- Keep recent messages at the end (optimal position)
- Avoid bloating the middle of context
- **Implication:** All strategies should preserve recent context

**From "Recursive Summarization" (Wang et al., 2023):**
- Summarization enables long-term consistency
- Works well for extended conversations
- **Implication:** Use summarization for long, high-value conversations

**From "MemGPT" (Packer et al., 2023):**
- Different strategies for different memory tiers
- Trade-offs between speed and quality
- **Implication:** Match strategy to use case requirements

**From Production Best Practices** (Anthropic, Vellum AI):
- Consider latency, cost, and quality trade-offs
- No one-size-fits-all solution
- **Implication:** Build a decision framework based on requirements


### Theory: Choosing the Right Strategy

**Decision Factors:**

1. **Quality Requirements**
   - High: Use summarization (preserves meaning)
   - Medium: Use priority-based (keeps important parts)
   - Low: Use truncation (fast and simple)

2. **Latency Requirements**
   - Fast: Use truncation or priority-based (no LLM calls)
   - Medium: Use priority-based with caching
   - Slow OK: Use summarization (requires LLM call)

3. **Conversation Length**
   - Short (<10 messages): No compression needed
   - Medium (10-30 messages): Truncation or priority-based
   - Long (>30 messages): Summarization recommended

4. **Cost Sensitivity**
   - High: Use truncation or priority-based (no LLM costs)
   - Medium: Use summarization with caching
   - Low: Use summarization freely

5. **Context Importance**
   - Critical: Use summarization (preserves all important info)
   - Important: Use priority-based (keeps high-value messages)
   - Less critical: Use truncation (simple and fast)


### Building the Decision Framework

Let's build a practical decision framework step-by-step.

#### Step 1: Define the available strategies


In [ ]:
from enum import Enum
from typing import Literal

class CompressionChoice(Enum):
    """Available compression strategies."""
    NONE = "none"
    TRUNCATION = "truncation"
    PRIORITY = "priority"
    SUMMARIZATION = "summarization"

print("✅ CompressionChoice enum defined")


#### Step 2: Create the decision function

This function takes your requirements and recommends the best strategy.


In [ ]:
def choose_compression_strategy(
    conversation_length: int,
    token_count: int,
    quality_requirement: Literal["high", "medium", "low"],
    latency_requirement: Literal["fast", "medium", "slow_ok"],
    cost_sensitivity: Literal["high", "medium", "low"] = "medium"
) -> CompressionChoice:
    """
    Decision framework for choosing compression strategy.

    Args:
        conversation_length: Number of messages in conversation
        token_count: Total token count
        quality_requirement: How important is quality? ("high", "medium", "low")
        latency_requirement: How fast must it be? ("fast", "medium", "slow_ok")
        cost_sensitivity: How sensitive to costs? ("high", "medium", "low")

    Returns:
        CompressionChoice: Recommended strategy
    """
    # No compression needed for short conversations
    if token_count < 2000 and conversation_length < 10:
        return CompressionChoice.NONE

    # Fast requirement = no LLM calls
    if latency_requirement == "fast":
        if quality_requirement == "high":
            return CompressionChoice.PRIORITY
        else:
            return CompressionChoice.TRUNCATION

    # High cost sensitivity = avoid LLM calls
    if cost_sensitivity == "high":
        return CompressionChoice.PRIORITY if quality_requirement != "low" else CompressionChoice.TRUNCATION

    # High quality + willing to wait = summarization
    if quality_requirement == "high" and latency_requirement == "slow_ok":
        return CompressionChoice.SUMMARIZATION

    # Long conversations benefit from summarization
    if conversation_length > 30 and quality_requirement != "low":
        return CompressionChoice.SUMMARIZATION

    # Medium quality = priority-based
    if quality_requirement == "medium":
        return CompressionChoice.PRIORITY

    # Default to truncation for simple cases
    return CompressionChoice.TRUNCATION

print("✅ Decision framework function defined")


### Demo 6: Test Decision Framework

Let's test the decision framework with various scenarios.

#### Step 1: Define test scenarios


In [ ]:
# Define test scenarios
scenarios = [
    # (length, tokens, quality, latency, cost, description)
    (5, 1000, "high", "fast", "medium", "Short conversation, high quality needed"),
    (15, 3000, "high", "slow_ok", "low", "Medium conversation, quality critical"),
    (30, 8000, "medium", "medium", "medium", "Long conversation, balanced needs"),
    (50, 15000, "high", "slow_ok", "medium", "Very long, quality important"),
    (100, 30000, "low", "fast", "high", "Extremely long, cost-sensitive"),
    (20, 5000, "medium", "fast", "high", "Medium length, fast and cheap"),
    (40, 12000, "high", "medium", "low", "Long conversation, quality focus"),
    (8, 1500, "low", "fast", "high", "Short, simple case"),
]

print(f"Defined {len(scenarios)} test scenarios")


#### Step 2: Run the decision framework on each scenario


In [ ]:
print("Decision Framework Test Results:")
print("=" * 120)
print(f"{'Scenario':<45} {'Length':<8} {'Tokens':<10} {'Quality':<10} {'Latency':<10} {'Cost':<8} {'Strategy'}")
print("-" * 120)

for length, tokens, quality, latency, cost, description in scenarios:
    strategy = choose_compression_strategy(length, tokens, quality, latency, cost)
    print(f"{description:<45} {length:<8} {tokens:<10,} {quality:<10} {latency:<10} {cost:<8} {strategy.value}")


#### Key Insights from the Decision Framework

**Pattern 1: Quality drives strategy choice**
- High quality + willing to wait → Summarization
- Medium quality → Priority-based
- Low quality → Truncation

**Pattern 2: Latency constraints matter**
- Fast requirement → Avoid summarization (no LLM calls)
- Slow OK → Summarization is an option

**Pattern 3: Cost sensitivity affects decisions**
- High cost sensitivity → Avoid summarization
- Low cost sensitivity → Summarization is preferred for quality

**Pattern 4: Conversation length influences choice**
- Short (<10 messages) → Often no compression needed
- Long (>30 messages) → Summarization recommended for quality

**Practical Recommendation:**
- Start with priority-based for most production use cases
- Use summarization for high-value, long conversations
- Use truncation for real-time, cost-sensitive scenarios


---

## 🏭 Part 6: Production Recommendations

Based on all the research and techniques we've covered, here are production-ready recommendations.


### Recommendation 1: For Most Applications (Balanced)

**Strategy:** Agent Memory Server with automatic summarization

**Configuration:**
- `message_threshold`: 20 messages
- `token_threshold`: 4000 tokens
- `keep_recent`: 4 messages
- `strategy`: "recent_plus_summary"

**Why:** Automatic, transparent, production-ready. Implements research-backed strategies (Liu et al., Wang et al., Packer et al.) with minimal code.

**Best for:** General-purpose chatbots, customer support, educational assistants


### Recommendation 2: For High-Volume, Cost-Sensitive (Efficient)

**Strategy:** Priority-based compression

**Configuration:**
- `max_tokens`: 2000
- Custom importance scoring
- No LLM calls

**Why:** Fast, cheap, no external dependencies. Preserves important messages without LLM costs.

**Best for:** High-traffic applications, real-time systems, cost-sensitive deployments


### Recommendation 3: For Critical Conversations (Quality)

**Strategy:** Manual summarization with review

**Configuration:**
- `token_threshold`: 5000
- Human review of summaries
- Store full conversation separately

**Why:** Maximum quality, human oversight. Critical for high-stakes conversations.

**Best for:** Medical consultations, legal advice, financial planning, therapy


### Recommendation 4: For Real-Time Chat (Speed)

**Strategy:** Truncation with sliding window

**Configuration:**
- `keep_recent`: 10 messages
- No summarization
- Fast response required

**Why:** Minimal latency, simple implementation. Prioritizes speed over context preservation.

**Best for:** Live chat, gaming, real-time collaboration tools


### General Guidelines

**Getting Started:**
1. Start with Agent Memory Server automatic summarization
2. Monitor token usage and costs in production
3. Adjust thresholds based on your use case

**Advanced Optimization:**
4. Consider hybrid approaches (truncation + summarization)
5. Always preserve critical information in long-term memory
6. Use the decision framework to adapt to different conversation types

**Monitoring:**
7. Track compression ratios and token savings
8. Monitor user satisfaction and conversation quality
9. A/B test different strategies for your use case


---

## 💪 Practice Exercises

Now it's your turn! Complete these exercises to reinforce your learning.


### Exercise 1: Implement Adaptive Compression Strategy

Create a strategy that automatically chooses between truncation and sliding window based on message token variance:

```python
class AdaptiveStrategy(CompressionStrategy):
    """
    Automatically choose between truncation and sliding window.

    Logic:
    - If messages have similar token counts → use sliding window (predictable)
    - If messages have varying token counts → use truncation (token-aware)
    """

    def __init__(self, window_size: int = 10):
        self.window_size = window_size
        self.truncation = TruncationStrategy()
        self.sliding_window = SlidingWindowStrategy(window_size)

    def compress(
        self,
        messages: List[ConversationMessage],
        max_tokens: int
    ) -> List[ConversationMessage]:
        """
        Choose strategy based on token variance.

        Steps:
        1. Calculate token count variance across messages
        2. If variance is low (similar sizes) → use sliding window
        3. If variance is high (varying sizes) → use truncation
        """
        # Your implementation here
        pass

# Test your implementation
adaptive = AdaptiveStrategy(window_size=6)
result = adaptive.compress(sample_conversation, max_tokens=800)
print(f"Adaptive strategy result: {len(result)} messages")
```

**Hint:** Calculate variance using `statistics.variance([msg.token_count for msg in messages])`. Use a threshold (e.g., 100) to decide.


### Exercise 2: Implement Hybrid Compression

Combine summarization + truncation for optimal results:

```python
async def compress_hybrid(
    messages: List[ConversationMessage],
    summarizer: ConversationSummarizer,
    max_tokens: int = 2000
) -> List[ConversationMessage]:
    """
    Hybrid compression: Summarize old messages, truncate if still too large.

    Steps:
    1. First, try summarization
    2. If still over budget, apply truncation to summary + recent messages
    3. Ensure we stay within max_tokens

    Args:
        messages: List of conversation messages
        summarizer: ConversationSummarizer instance
        max_tokens: Maximum token budget

    Returns:
        Compressed messages within token budget
    """
    # Your implementation here
    pass

# Test your implementation
hybrid_result = await compress_hybrid(sample_conversation, summarizer, max_tokens=1000)
print(f"Hybrid compression: {len(hybrid_result)} messages, {sum(m.token_count for m in hybrid_result)} tokens")
```

**Hint:** Use `summarizer.compress_conversation()` first, then apply truncation if needed.


### Exercise 3: Quality Comparison

Test all compression strategies and compare quality:

```python
async def compare_compression_quality(
    messages: List[ConversationMessage],
    test_query: str = "What courses did we discuss?"
) -> Dict[str, Any]:
    """
    Compare compression strategies by testing reference resolution.

    Steps:
    1. Compress using each strategy
    2. Try to answer test_query using compressed context
    3. Compare quality of responses
    4. Measure token savings

    Args:
        messages: Original conversation
        test_query: Question to test reference resolution

    Returns:
        Dictionary with comparison results
    """
    # Your implementation here
    # Test if the agent can still answer questions after compression
    pass

# Test your implementation
quality_results = await compare_compression_quality(sample_conversation)
print("Quality Comparison Results:")
for strategy, results in quality_results.items():
    print(f"{strategy}: {results}")
```

**Hint:** Use the LLM to answer the test query with each compressed context and compare responses.


### Exercise 4: Custom Importance Scoring

Improve the `calculate_importance()` function with domain-specific logic:

```python
def calculate_importance_enhanced(msg: ConversationMessage) -> float:
    """
    Enhanced importance scoring for course advisor conversations.

    Add scoring for:
    - Specific course codes (CS401, MATH301, etc.) - HIGH
    - Prerequisites and requirements - HIGH
    - Student preferences and goals - HIGH
    - Questions - MEDIUM
    - Confirmations and acknowledgments - LOW
    - Greetings and small talk - VERY LOW

    Returns:
        Importance score (0.0 to 5.0)
    """
    # Your implementation here
    pass

# Test your implementation
for msg in sample_conversation[:5]:
    score = calculate_importance_enhanced(msg)
    print(f"Score: {score:.1f} - {msg.content[:60]}...")
```

**Hint:** Use regex to detect course codes, check for question marks, look for keywords.


### Exercise 5: Production Configuration

Configure Agent Memory Server for your specific use case:

```python
# Scenario: High-volume customer support chatbot
# Requirements:
# - Handle 1000+ conversations per day
# - Average conversation: 15-20 turns
# - Cost-sensitive but quality important
# - Response time: <2 seconds

# Your task: Choose appropriate configuration
production_config = {
    "message_threshold": ???,  # When to trigger summarization
    "token_threshold": ???,    # Token limit before summarization
    "keep_recent": ???,        # How many recent messages to keep
    "strategy": ???,           # Which strategy to use
}

# Justify your choices:
print("Configuration Justification:")
print(f"message_threshold: {production_config['message_threshold']} because...")
print(f"token_threshold: {production_config['token_threshold']} because...")
print(f"keep_recent: {production_config['keep_recent']} because...")
print(f"strategy: {production_config['strategy']} because...")
```

**Hint:** Consider the trade-offs between cost, quality, and latency for this specific scenario.


---

## 📝 Summary

### **What You Learned:**

1. ✅ **Research Foundations**
   - "Lost in the Middle" (Liu et al., 2023): U-shaped performance, non-uniform degradation
   - "Recursive Summarization" (Wang et al., 2023): Long-term dialogue memory
   - "MemGPT" (Packer et al., 2023): Hierarchical memory management
   - Production best practices from Anthropic and Vellum AI

2. ✅ **The Long Conversation Problem**
   - Token limits, cost implications, performance degradation
   - Why unbounded growth is unsustainable
   - Quadratic cost growth without management
   - Why larger context windows don't solve the problem

3. ✅ **Conversation Summarization**
   - What to preserve vs. compress
   - When to trigger summarization (token/message thresholds)
   - Building summarization step-by-step (functions → class)
   - LLM-based intelligent summarization

4. ✅ **Three Compression Strategies**
   - **Truncation:** Fast, simple, loses context
   - **Priority-based:** Balanced, intelligent, no LLM calls
   - **Summarization:** High quality, preserves meaning, requires LLM
   - Trade-offs between speed, quality, and cost

5. ✅ **Agent Memory Server Integration**
   - Automatic summarization configuration
   - Transparent memory management
   - Production-ready solution implementing research findings
   - Configurable thresholds and strategies

6. ✅ **Decision Framework**
   - How to choose the right strategy
   - Factors: quality, latency, cost, conversation length
   - Production recommendations for different scenarios
   - Hybrid approaches for optimal results

### **What You Built:**

- ✅ `ConversationSummarizer` class for intelligent summarization
- ✅ Three compression strategy implementations (Truncation, Priority, Summarization)
- ✅ Decision framework for strategy selection
- ✅ Production configuration examples
- ✅ Comparison tools for evaluating strategies
- ✅ Token counting and cost analysis tools

### **Key Takeaways:**

💡 **"Conversations grow unbounded without management"**
- Every turn adds tokens and cost
- Eventually you'll hit limits
- Costs grow quadratically (each turn includes all previous messages)

💡 **"Summarization preserves meaning while reducing tokens"**
- Use LLM to create intelligent summaries
- Keep recent messages for immediate context
- Store important facts in long-term memory

💡 **"Choose strategy based on requirements"**
- Quality-critical → Summarization
- Speed-critical → Truncation or Priority-based
- Balanced → Agent Memory Server automatic
- Cost-sensitive → Priority-based

💡 **"Agent Memory Server handles this automatically"**
- Production-ready solution
- Transparent to your application
- Configurable for your needs
- No manual intervention required

### **Connection to Context Engineering:**

This notebook completes the **Conversation Context** story from Section 1:

1. **Section 1:** Introduced the 4 context types, including Conversation Context
2. **Section 3, NB1:** Implemented working memory for conversation continuity
3. **Section 3, NB2:** Integrated memory with RAG for stateful conversations
4. **Section 3, NB3:** Managed long conversations with summarization and compression ← You are here

**Next:** Section 4 will show how agents can actively manage their own memory using tools!

### **Next Steps:**

**Section 4: Tools and Agents**
- Build agents that actively manage their own memory
- Implement memory tools (store, search, retrieve)
- Use LangGraph for agent workflows
- Let the LLM decide when to summarize

**Section 5: Production Optimization**
- Performance measurement and monitoring
- Hybrid retrieval strategies
- Semantic tool selection
- Quality assurance and validation

---

## 🔗 Resources

### **Documentation:**
- [Agent Memory Server](https://github.com/redis/agent-memory-server) - Production memory management
- [Agent Memory Client](https://pypi.org/project/agent-memory-client/) - Python client library
- [LangChain Memory](https://python.langchain.com/docs/modules/memory/) - Memory patterns
- [OpenAI Tokenizer](https://platform.openai.com/tokenizer) - Token counting tool
- [tiktoken](https://github.com/openai/tiktoken) - Fast token counting library

### **Research Papers:**
- **[Lost in the Middle: How Language Models Use Long Contexts](https://arxiv.org/abs/2307.03172)** - Liu et al. (2023). Shows U-shaped performance curve and non-uniform degradation in long contexts.
- **[Recursively Summarizing Enables Long-Term Dialogue Memory in Large Language Models](https://arxiv.org/abs/2308.15022)** - Wang et al. (2023). Demonstrates recursive summarization for long conversations.
- **[MemGPT: Towards LLMs as Operating Systems](https://arxiv.org/abs/2310.08560)** - Packer et al. (2023). Introduces hierarchical memory management and virtual context.
- [Retrieval-Augmented Generation](https://arxiv.org/abs/2005.11401) - RAG fundamentals
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762) - Transformer architecture and context windows

### **Industry Resources:**
- **[How Should I Manage Memory for my LLM Chatbot?](https://www.vellum.ai/blog/how-should-i-manage-memory-for-my-llm-chatbot)** - Vellum AI. Practical insights on memory management trade-offs.
- **[Lost in the Middle Paper Reading](https://arize.com/blog/lost-in-the-middle-how-language-models-use-long-contexts-paper-reading/)** - Arize AI. Detailed analysis and practical implications.
- **[Effective Context Engineering for AI Agents](https://www.anthropic.com/engineering/effective-context-engineering-for-ai-agents)** - Anthropic. Production best practices.


### **Tools and Libraries:**
- **Redis:** Vector storage and memory backend
- **Agent Memory Server:** Dual-memory architecture with automatic summarization
- **LangChain:** LLM interaction framework
- **LangGraph:** State management and agent workflows
- **OpenAI:** GPT-4o for generation and summarization
- **tiktoken:** Token counting for cost estimation

---

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

**Redis University - Context Engineering Course**

**🎉 Congratulations!** You've completed Section 3: Memory Architecture!

You now understand how to:
- Build memory systems for AI agents
- Integrate working and long-term memory
- Manage long conversations with summarization
- Choose the right compression strategy
- Configure production-ready memory management

**Ready for Section 4?** Let's build agents that actively manage their own memory using tools!

---

